In [1]:
# import libraries
import pandas as pd
import os
from IPython.display import display


In [2]:
# Global parameters for cleaning & stitching
years_a = [2006, 2007, 2008]
years_b = [2014, 2015, 2016]

In [3]:
#Setup intake parameters (file_names, ...)
file_prefix_list_a = ['ehact', 'ehchild', 'ehresp']
file_prefix_list_b = ['ehact', 'ehresp']

# populate list of files to read
# Note: 2014-2016 may not have ehchild data.  TODO - check ATUS available data.
params = {}
for year in years_a:
    year = str(year)
    params[year+'_in'] = [x + '_' + year + '.dat' for x in file_prefix_list_a]
    params[year+'_out'] = year + '_clean.csv'
    
for year in years_b:
    year = str(year)
    params[year+'_in'] = [x + '_' + year + '.dat' for x in file_prefix_list_b]
    params[year+'_out'] = year + '_clean.csv'

display(params)

{'2006_in': ['ehact_2006.dat', 'ehchild_2006.dat', 'ehresp_2006.dat'],
 '2006_out': '2006_clean.csv',
 '2007_in': ['ehact_2007.dat', 'ehchild_2007.dat', 'ehresp_2007.dat'],
 '2007_out': '2007_clean.csv',
 '2008_in': ['ehact_2008.dat', 'ehchild_2008.dat', 'ehresp_2008.dat'],
 '2008_out': '2008_clean.csv',
 '2014_in': ['ehact_2014.dat', 'ehresp_2014.dat'],
 '2014_out': '2014_clean.csv',
 '2015_in': ['ehact_2015.dat', 'ehresp_2015.dat'],
 '2015_out': '2015_clean.csv',
 '2016_in': ['ehact_2016.dat', 'ehresp_2016.dat'],
 '2016_out': '2016_clean.csv'}

In [4]:
# Check present working directory for filenames
path = os.getcwd() 
dir_list = os.listdir(path) 
  
print("Files and directories in '", path, "' :")  
for item in dir_list:
    print(item)

Files and directories in ' C:\Users\vince\Desktop\lambdaschool_temp\4.portfolioprojects\eat_health ' :
.ipynb_checkpoints
2006_clean.csv
archive
atususersguide.pdf
ehact_2006.dat
ehact_2007.dat
ehact_2008.dat
ehact_2014.dat
ehact_2015.dat
ehact_2016.dat
ehchild_2006.dat
ehchild_2007.dat
ehchild_2008.dat
ehmintcodebk1416.pdf
ehmintcodebk_06-08.pdf
ehresp_2006.dat
ehresp_2007.dat
ehresp_2008.dat
ehresp_2014.dat
ehresp_2015.dat
ehresp_2016.dat
read_clean.ipynb
stitch.ipynb


In [158]:
# Intake data

# Filter Control
year = 2014

input_file_list = params[str(year)+'_in']

working_data = {}
for input_file in input_file_list:
    working_data[input_file] = pd.read_csv(input_file)

for key, item in working_data.items():
    display(key, item.head(3))

'ehact_2014.dat'

,TUCASEID,TUACTIVITY_N,EUEATSUM,EUEDUR,EUEDUR24
0,20140101140007,1,-1,-1,-1
1,20140101140007,2,-1,-1,-1
2,20140101140007,3,-1,-1,-1


'ehresp_2014.dat'

,TUCASEID,TULINENO,EEINCOME1,ERBMI,ERHHCH,ERINCOME,ERSPEMCH,ERTPREAT,ERTSEAT,ETHGT,...,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20140101140007,1,-2,33.2,1,-1,-1,30,2,0,...,1,2,1,-1,2,1,2,170,1,2
1,20140101140011,1,1,22.7,3,1,-1,45,14,0,...,1,2,1,-1,1,2,2,128,2,0
2,20140101140028,1,2,49.4,3,5,-1,60,0,0,...,-1,-1,2,2,-1,-1,-1,270,2,12


In [159]:
# Output Prep
output_file_name = params[str(year)+'_out']
output_file_name

'2014_clean.csv'

## Interpret/Transform: Eating & Health Activity File (EHACT)
**Identify data to keep.  Drop per column name match criteria (see below)**

--2006 Thru 2008--
* TUCASEID: case_id
* TUACTIVITY_N: activity_number
* EUEATSUM: eating_during_activity
* EUDRKSUM: drinking_during_activity (non-water implied for analysis.  not included in new var name)
* EUEDUR: eating_duration
* EUDDUR: drinking_duration
* EUEDUR24: eating_duration_24h
* EUDDUR24: drinking_druation_24h

**Note, EHACT does not include demographic information needed for application**

For ease of client understanding and analysis (as well as applicationd development), EUEDUR24/EUDDUR24 will be used in lieu of EUEDUR and EUDDUR.

In [160]:
ehact = working_data[input_file_list[0]].drop(columns=['EUEDUR', 'EUDDUR']).copy()

KeyError: "['EUDDUR'] not found in axis"

In [118]:
ehact_columns = [
    'case_id', 'activity_number', 'eating_during_activity', 
    'drinking_during_activity', 'eating_duration_24', 'drinking_duration_24'
]
ehact.columns = ehact_columns
display(ehact.head(2))

,case_id,activity_number,eating_during_activity,drinking_during_activity,eating_duration_24,drinking_duration_24
0,20080101080102,1,-1,-1,-1,-1
1,20080101080102,2,-1,-1,-1,-1


### Deal with missing values or unusable data for our use case

**Acceptable values:

* eating_during_activity or drinking_during_activity are (0,1)
* eating_duration_24, drinking_duration_24 (>1 & <1440)

In [119]:
import numpy as np

def allocation_check(x):
    '''
    Check allocation column for acceptable value and apply np.nan/0 to missing data.
    ATUS allocation values:
        0 Value – no change
        10 Value to value
        11 Blank to value
        12 Don’t know to value
        13 Refused to value
        21 Blank to longitudinal value
        22 Don’t know to longitudinal value
        23 Refused to longitudinal value
        30 Value to allocated longitudinal value (unused)
        31 Blank to allocated longitudinal value (unused)
        32 Don’t know to allocated longitudinal value (unused)
        33 Refused to allocated longitudinal value (unused)
        40 Value to allocated value
        41 Blank to allocated value
        42 Don’t know to allocated value
        43 Refused to allocated value
        71 Refused to assigned from respondent's earnings
        72 Don't know to assigned from earnings of respondent's spouse or unmarried partner
        73 Don't know to assigned from earnings of respondent's other household members (not spouse or unmarried partner)
        74 Don't know to assigned from earnings of respondent and respondent's spouse or unmarried partner
        75 Don't know to assigned from earnings of respondent and respondent's other household members (not spouse or unmarried partner)
        76 Don't know to assigned from earnings of respondent's spouse or unmarried partner and respondent's other household members
        77 Don't know to assigned from earnings of respondent, respondent's spouse or unmarried partner, and respondent's other household members
        81 Refused to assigned from respondent's earnings
        82 Refused to assigned from earnings of respondent's spouse or unmarried partner
        83 Refused to assigned from earnings of respondent's other household members (not spouse or unmarried partner)
        84 Refused to assigned from earnings of respondent and respondent's spouse or unmarried partner
        85 Refused to assigned from earnings of respondent and respondent's other household members (not spouse or unmarried partner)
        86 Refused to assigned from earnings of respondent's spouse or unmarried partner and respondent's other household members
        87 Refused to assigned from earnings of respondent, respondent's spouse or unmarried partner, and respondent's other household members
    
    '''
    value_present_allocations = [0, 10, 11, 12, 13, 21, 22, 23, 30, 31, 32, 33,
                                 40, 41, 42, 43, 71, 72, 73, 74, 75, 76, 77,
                                81, 82, 83, 84, 85, 86, 87]
    if x in value_present_allocations:
        return 1
    else:
        return 0
    

In [120]:
import numpy as np

def value_check(x, minima, maxima, nan_opt):
    if x >= minima and x <= maxima:
        return x
    else:
        if nan_opt:
            return np.nan
        else:
            return 0

In [121]:
ehact.eating_duration_24.value_counts().head()

-1     131019
 10      2198
 5       2044
 15      1526
 30      1213
Name: eating_duration_24, dtype: int64

In [122]:
# Convert -1 to 0 for multiplication to eating_duration_24
ehact['eating_during_activity'] = ehact.eating_during_activity.apply(value_check, args=(0,2,False))
ehact['drinking_during_activity'] = ehact.drinking_during_activity.apply(value_check, args=(0,2,False))

ehact.eating_during_activity.value_counts()

0    131019
1     11297
Name: eating_during_activity, dtype: int64

In [123]:
# Multiply binary existance (eating_during_activity) by duration to standardize missing values as 0.
# Then keep only positive numbers.
# Drop binary existance columns
ehact['eating_duration_24'] = ehact['eating_duration_24'] * ehact['eating_during_activity']
ehact['drinking_duration_24'] = ehact['drinking_duration_24'] * ehact['drinking_during_activity']

ehact = ehact.drop(columns=['eating_during_activity', 'drinking_during_activity'])

ehact['eating_duration_24'] = ehact['eating_duration_24'].apply(value_check, args=(1,1440, True))
ehact['drinking_duration_24'] = ehact['drinking_duration_24'].apply(value_check, args=(1,1440, True))

In [124]:
display(ehact.case_id.nunique(), ehact.shape)

6923

(142316, 4)

In [125]:
# Groupby case_id to create features secondary_eating_24, secondary_drinking_24
# Turn activity_numbers into list/case_id so summation retains all activities.
# Store in new value activity_list

In [126]:
# Return rows with information
ehact_condensed = ehact[ehact['eating_duration_24'].notna() | ehact['drinking_duration_24'].notna()]
ehact_condensed.head()

,case_id,activity_number,eating_duration_24,drinking_duration_24
3,20080101080102,4,5.0,5.0
21,20080101080102,22,20.0,20.0
25,20080101080102,26,NaN,30.0
32,20080101080116,6,30.0,30.0
33,20080101080116,7,NaN,30.0


In [127]:
# Get a list of all ids to query
active_ids = ehact_condensed.case_id.unique().tolist()

# Query each id and return list of activities stored in dictionary
activity_dict = {}
for active_id in active_ids:
    activity_dict[active_id] = ','.join(ehact_condensed.loc[ehact_condensed.case_id==active_id].activity_number.astype(str).tolist())

In [128]:
# Summation by case_id.  groupby automatically re|sets index to case_id
ehact_condensed = ehact_condensed[['case_id', 'eating_duration_24', 'drinking_duration_24']].groupby('case_id').sum()
ehact_condensed.head()

,eating_duration_24,drinking_duration_24
case_id,,
20080101080102,25.0,55.0
20080101080116,30.0,135.0
20080101080145,5.0,5.0
20080101080164,15.0,115.0
20080101080504,30.0,0.0


In [129]:
# Add back activity list.  All this work is the equivalent of unstacking...well ok
list_activities = pd.DataFrame.from_dict(activity_dict, orient='index')
list_activities.columns = ['activity_list']
list_activities.index = list_activities.index.set_names('case_id')
list_activities.head()

,activity_list
case_id,
20080101080102,"4,22,26"
20080101080116,"6,7,8,11"
20080101080145,"8,22"
20080101080164,"3,26,30"
20080101080504,"4,5"


In [130]:
# Merge condensed activity list and reformatted activities
ehact_condensed = ehact_condensed.join(list_activities)
ehact_condensed.head()

,eating_duration_24,drinking_duration_24,activity_list
case_id,,,
20080101080102,25.0,55.0,"4,22,26"
20080101080116,30.0,135.0,"6,7,8,11"
20080101080145,5.0,5.0,"8,22"
20080101080164,15.0,115.0,"3,26,30"
20080101080504,30.0,0.0,"4,5"


## Interpret/Transform: Eating & Health Child Responses

Analysis of the children directly is not intended.  The goal here is to profile the household and append to the eventual final table for modeling.

**Identify data to keep. Drop per column name match criteria (see below)**

--2006 Thru 2008--
* TUCASEID: case_id
* TULINENO: person_number
* EXLCH: ate_school_lunch_allocation
* EELCH: ate_school_lunch
* EEBRK: ate_school_breakfast
* EXBRK: ate_school_breakfast_allocation

### Engineered Features
**One row per case_id**

* total_children
* num_assisted_meals_children

In [131]:
ehchild_columns = [
    'case_id', 'person_number', 'ate_school_lunch_allocation',
    'ate_school_lunch', 'ate_school_breakfast', 'ate_school_breakfast_allocation' 
]
ehchild = working_data[input_file_list[1]].copy()
ehchild.columns = ehchild_columns
display(ehchild.head(2))

,case_id,person_number,ate_school_lunch_allocation,ate_school_lunch,ate_school_breakfast,ate_school_breakfast_allocation
0,20080101080102,3,0,2,2,0
1,20080101080102,4,0,2,2,0


In [132]:
# Check for valid allocation codes
ehchild['ate_school_lunch_allocation'] = ehchild['ate_school_lunch_allocation'].apply(allocation_check)

In [133]:
# Check for valid allocation codes
ehchild['ate_school_breakfast_allocation'] = ehchild['ate_school_breakfast_allocation'].apply(allocation_check)

In [134]:
# Set invalid data to zero where appropriate
ehchild['ate_school_lunch'] = ehchild.ate_school_lunch * ehchild.ate_school_lunch_allocation
ehchild['ate_school_breakfast'] = ehchild.ate_school_breakfast * ehchild.ate_school_breakfast_allocation

In [135]:
# Drop allocation columns
ehchild = ehchild.drop(columns=['ate_school_lunch_allocation', 'ate_school_breakfast_allocation'])

In [136]:
# Convert 2's in eating to zero, leave 1s as mean provided
ehchild['ate_school_breakfast'] = ehchild.ate_school_breakfast % 2
ehchild['ate_school_lunch'] = ehchild.ate_school_lunch % 2

In [137]:
# Set all person_numbers to 1 for simple summation to number children
ehchild['person_number'] = 1

In [138]:
# Group by case_id and sum meals and children in household.  Rename columns
ehchild_condensed = ehchild.groupby('case_id').sum()
ehchild_condensed['assisted_meals'] = ehchild_condensed.ate_school_breakfast + ehchild_condensed.ate_school_lunch
ehchild_condensed = ehchild_condensed.drop(columns=['ate_school_lunch', 'ate_school_breakfast'])
ehchild_condensed.columns = ['num_children', 'assisted_meals']
ehchild_condensed = ehchild_condensed.drop(columns='num_children') #duplicate data.  See ehresp.
ehchild_condensed.head(3)

,assisted_meals
case_id,
20080101080102,2
20080101080116,1
20080101080119,8


In [139]:
# Combine ehchild_condensed and ehact_condensed --> NOT WORKING CORRECTLY.  GROUPBY EHACT
df_join_1 = ehact_condensed.join(ehchild_condensed)
df_join_1 = df_join_1.drop(columns=['eating_duration_24', 'drinking_duration_24']) # repeated in response with allocation.  process retained for future evaluation
df_join_1.head(3)

,activity_list,assisted_meals
case_id,,
20080101080102,"4,22,26",2.0
20080101080116,"6,7,8,11",1.0
20080101080145,"8,22",NaN


## Interpret/Transform: Response Data

This is the primary data file for each year, containing targets BMI and overall health

**Identify data to keep. Drop per column name match criteria (see below)**

--2006 Thru 2008--
* TUCASEID: case_id
* (DROP) TULINENO
* EUFSP: snap_benefits (1: yes; 2: no recode to --> 0: no; 1: yes)
* EUGENHTH: general_health (1: excellent; 2: very good; 3: good; 4: fair; 5: poor -> Recode, Invert)
* EUHGT: height_in
* EUINCOME2: income_130_pov (recode ordinal relationship, 0 <= 130%, 1 > 130%)
* ETWGT: topcode_weight (0: no code, val; 1: topcode, val=max 330lb; 2: bottomcode, val=min 98lb) -- See engineered features
* ETHGT: topcode_height (0: no code, val; 1: topcode, val=max 77in; 2: bottomcode, val=min 56in)
* (DROP) EUINCLVL: context - year of income for poverty threshold calculation in EUINCOME2.  Difference within yearly datasets not considered here.
* EUFINLWGT: statistical weight - may need to be normalized per dataset for population level stats over time
* (DROP) ERSPEMCH: Change in partner/spouse employment from previous CPS study (Not considered in Version 1 as CPS data not included in model)
* ERC19NUM: num_children
* EXINCOME1: EEINCOME1 allocation flag -> modified via allocation_check() to clean EEINCOME1
* (DROP) ERINCOME: relationship between income and poverty threshold (derived feature not considered in Version 1)
* EEINCOME1: income_185_pov (recode ordinal relationship, 0 <= 185%, 1 > 185%)
* EUPRPMEL: meal_preparer (1: yes; 2: no; 3: split with others) **Consider categorical encoding**
* EUGROSHP: grocery_shopper (1: yes; 2: no; 3: split equally) **Consider categorical encoding**
* ERTSEAT: secondary_eating_duration 
* ERTSDRK: secondary_drinking_duration
* ERTPREAT: primary_eating_duration
* EUDRINK: secondary_drinking_day_prior
* EUWGT: weight_lb
* EUEAT: seondary_eating_day_prior (1: yes; 2: no recode to --> 0: no; 1: yes)
* (DROP) ERHHCH: change in household from CPS survey (not considered in v1)
* ERBMI: bmi
* (DROP) EXSCLBRK: allocation flag for EESCLBRK (v1: represented via assisted meals)
* (DROP) EESCLBRK:children ate prepared breakfast by outside institution (v1: represented via assisted meals)
* (DROP) EXSCLLCH: allocation flag for EESCLLCH (v1: represented via assited meals)
* (DROP) EESCLLCH: children ate prepared lunch by outside instituion (v1: represented via assisted meals)
* (DROP) ERPLWC: proportion_children_assisted_lunch
* (DROP) ERPBWC: proportion_children_assisted_breakfast
* EREATAD: secondary_eating_all_day (1: yes; 2: no recode to --> 0: no; 1: yes)
* ERDRKAD: secondary_drinking_all_day (1: yes; 2: no recode to --> 0: no; 1: yes)


### Engineered Features
**One row per case_id**
* weight_modified: topcode/bottom code re-calculated (v2: 1std from min to significantly distance from values near max/min)
* height_modified: topcode/bottom code re-calculated (v2: 1std from min to significantly distance from values near max/min)

In [140]:
ehresp = working_data[input_file_list[2]].copy()
ehresp.head(2)

,TUCASEID,TULINENO,EUFSP,EUGENHTH,EUHGT,EUINCOME2,ETWGT,ETHGT,EUINCLVL,EUFINLWGT,...,ERHHCH,ERBMI,EXSCLBRK,EESCLBRK,EXSCLLCH,EESCLLCH,ERPLWC,ERPBWC,EREATAD,ERDRKAD
0,20080101080013,1,2,3,67,-1,0,0,3,1.739920e+07,...,2,28.2,-1,-1,-1,-1,-1.0,-1.0,2,2
1,20080101080015,1,2,4,68,-2,0,0,3,5.435524e+06,...,3,30.4,-1,-1,-1,-1,-1.0,-1.0,2,2


In [141]:
display(ehresp.columns)

Index(['TUCASEID', 'TULINENO', 'EUFSP', 'EUGENHTH', 'EUHGT', 'EUINCOME2',
       'ETWGT', 'ETHGT', 'EUINCLVL', 'EUFINLWGT', 'ERSPEMCH', 'ERC19NUM',
       'EXINCOME1', 'ERINCOME', 'EEINCOME1', 'EUPRPMEL', 'EUGROSHP', 'ERTSEAT',
       'ERTSDRK', 'ERTPREAT', 'EUDRINK', 'EUWGT', 'EUEAT', 'ERHHCH', 'ERBMI',
       'EXSCLBRK', 'EESCLBRK', 'EXSCLLCH', 'EESCLLCH', 'ERPLWC', 'ERPBWC',
       'EREATAD', 'ERDRKAD'],
      dtype='object')

In [142]:
resp_columns_keep = [
        'TUCASEID', 'EUFSP', 'EUGENHTH', 'EUHGT', 'EUINCOME2', 'ETWGT', 'ETHGT', 'EUFINLWGT', 'ERC19NUM',
        'EXINCOME1', 'EEINCOME1', 'EUPRPMEL', 'EUGROSHP', 'ERTSEAT', 'ERTSDRK', 'ERTPREAT', 'EUDRINK', 
        'EUWGT', 'EUEAT', 'ERBMI', 'EREATAD', 'ERDRKAD'
]

name_map = {
    'TUCASEID': 'case_id', 
    'EUFSP': 'snap_benefits',
    'EUGENHTH': 'general_health',
    'EUHGT': 'height_in',
    'EUINCOME2': 'income_130_pov',
    'ETWGT': 'topcode_weight',
    'ETHGT': 'topcode_height',
    'EUFINLWGT': 'statistical_weight',
    'ERC19NUM': 'num_children', 
    'EXINCOME1': 'allocation_flag_income_185',
    'EEINCOME1': 'income_185_pov', 
    'EUPRPMEL': 'meal_preparer', 
    'EUGROSHP': 'grocery_shopper',
    'ERTSEAT': 'secondary_eating_duration', 
    'ERTSDRK': 'secondary_drinking_duration', 
    'ERTPREAT': 'primary_eating_duration', 
    'EUDRINK': 'secondary_drinking_day_prior',
    'EUWGT': 'weight_lb', 
    'EUEAT': 'secondary_eating_day_prior', 
    'ERBMI': 'bmi', 
    'EREATAD': 'secondary_eating_all_day', 
    'ERDRKAD': 'secondary_drinking_all_day',
}

ehresp = ehresp[resp_columns_keep].rename(columns=name_map)

In [143]:
# review changes
display(ehresp.columns, ehresp.shape, ehresp.case_id.nunique(), ehresp.head(2))

Index(['case_id', 'snap_benefits', 'general_health', 'height_in',
       'income_130_pov', 'topcode_weight', 'topcode_height',
       'statistical_weight', 'num_children', 'allocation_flag_income_185',
       'income_185_pov', 'meal_preparer', 'grocery_shopper',
       'secondary_eating_duration', 'secondary_drinking_duration',
       'primary_eating_duration', 'secondary_drinking_day_prior', 'weight_lb',
       'secondary_eating_day_prior', 'bmi', 'secondary_eating_all_day',
       'secondary_drinking_all_day'],
      dtype='object')

(12708, 22)

12708

,case_id,snap_benefits,general_health,height_in,income_130_pov,topcode_weight,topcode_height,statistical_weight,num_children,allocation_flag_income_185,...,grocery_shopper,secondary_eating_duration,secondary_drinking_duration,primary_eating_duration,secondary_drinking_day_prior,weight_lb,secondary_eating_day_prior,bmi,secondary_eating_all_day,secondary_drinking_all_day
0,20080101080013,2,3,67,-1,0,0,1.739920e+07,0,0,...,1,0,0,20,2,180,2,28.2,2,2
1,20080101080015,2,4,68,-2,0,0,5.435524e+06,0,2,...,2,0,0,5,2,200,2,30.4,2,2


In [144]:
# set index to case_id - assert that case_id's must be unique via shape[0] = case_id.nunique()
assert ehresp.shape[0] == ehresp.case_id.nunique()
ehresp.index = ehresp.case_id
ehresp = ehresp.drop(columns='case_id')
ehresp.index.set_names('case_id')

Int64Index([20080101080013, 20080101080015, 20080101080088, 20080101080102,
            20080101080116, 20080101080119, 20080101080134, 20080101080145,
            20080101080164, 20080101080504,
            ...
            20081212082401, 20081212082428, 20081212082436, 20081212082455,
            20081212082457, 20081212082468, 20081212082480, 20081212082481,
            20081212082491, 20081212082492],
           dtype='int64', name='case_id', length=12708)

In [145]:
# Check SNAP benefits for valid data
# EUFSP: snap_benefits (1: yes; 2: no recode to --> 0: no; 1: yes)
ehresp.snap_benefits = abs(ehresp.snap_benefits - 2)

# Pass through value check
ehresp.snap_benefits = ehresp.snap_benefits.apply(value_check, args = (0,1,True))
display(ehresp.snap_benefits.unique())

array([ 0.,  1., nan])

In [146]:
# Recode general_health
# general_health (current = 1: excellent; 2: very good; 3: good; 4: fair; 5: poor || inverted = 1. poor...5. excellent)
ehresp.general_health = abs(ehresp.general_health-5)+1

In [147]:
# Topcode/Bottomcode adjustments
# Not completing in v1.  Columns dropped
ehresp = ehresp.drop(columns=['topcode_weight', 'topcode_height',])

In [148]:
#EUPRPMEL: meal_preparer (1: yes; 2: no; 3: split with others) Consider categorical encoding
#EUGROSHP: grocery_shopper (1: yes; 2: no; 3: split equally) Consider categorical encoding

# First, clean meta values from series
ehresp.meal_preparer = ehresp.meal_preparer.apply(value_check, args=(1,3,False))
ehresp.grocery_shopper = ehresp.grocery_shopper.apply(value_check, args=(1,3,False))

# There is an ordinal meaning in these categories. The responder has assistance in 2, 3 but not in 1.
# 2, 'no', implies 100% assistance in meal and grocer activities. 3 implies ~50% or greater assistance.
# Here, I revolve 2 and 3 to better associate implicit assistance

def flip_num(x):
    lookup = {
        1: 1,
        2: 3,
        3: 2,
        0: np.nan,
    }
    return lookup[x]

ehresp.meal_preparer = ehresp.meal_preparer.apply(flip_num)
ehresp.grocery_shopper = ehresp.grocery_shopper.apply(flip_num)

In [149]:
# EUDRINK: secondary_drinking_day_prior
# EUEAT: seondary_eating_day_prior (1: yes; 2: no recode to --> 0: no; 1: yes)

# First, clean meta values from series
ehresp.secondary_drinking_day_prior = ehresp.secondary_drinking_day_prior.apply(value_check, args=(1,2,False))
ehresp.secondary_eating_day_prior = ehresp.secondary_eating_day_prior.apply(value_check, args=(1,2,False))

# Recode
ehresp.secondary_drinking_day_prior = abs(ehresp.secondary_drinking_day_prior - 2)
ehresp.secondary_eating_day_prior = abs(ehresp.secondary_eating_day_prior - 2)

# Set meta values to NaN
ehresp.secondary_drinking_day_prior = ehresp.secondary_drinking_day_prior.apply(value_check, args=(0,1,True))
ehresp.secondary_eating_day_prior = ehresp.secondary_eating_day_prior.apply(value_check, args=(0,1,True))

display(ehresp.secondary_drinking_day_prior.unique(), ehresp.secondary_eating_day_prior.unique())

array([ 0.,  1., nan])

array([ 0.,  1., nan])

In [150]:
# EREATAD: secondary_eating_all_day (1: yes; 2: no recode to --> 0: no; 1: yes)
# ERDRKAD: secondary_drinking_all_day (1: yes; 2: no recode to --> 0: no; 1: yes)

# Cycle values to 0,1
ehresp.secondary_eating_all_day = abs(ehresp.secondary_eating_all_day - 2)
ehresp.secondary_drinking_all_day = abs(ehresp.secondary_drinking_all_day - 2)

# Set meta values to NaN
ehresp.secondary_eating_all_day = ehresp.secondary_eating_all_day.apply(value_check, args=(0,1,True))
ehresp.secondary_drinking_all_day = ehresp.secondary_drinking_all_day.apply(value_check, args=(0,1,True))

In [151]:
display(ehresp.secondary_eating_all_day.unique(), ehresp.secondary_drinking_all_day.unique())

array([0, 1], dtype=int64)

array([0, 1], dtype=int64)

In [152]:
display(ehresp.shape[0], ehresp.secondary_eating_all_day.isna().sum(), ehresp.secondary_drinking_all_day.isna().sum())

12708

0

0

In [153]:
# Check for valid allocation codes: for income_185_pov
ehresp.allocation_flag_income_185 = ehresp.allocation_flag_income_185.apply(allocation_check)

# Multiply through to zero out any invalid data
ehresp.income_185_pov = ehresp.income_185_pov * ehresp.allocation_flag_income_185
ehresp = ehresp.drop(columns='allocation_flag_income_185')

# Pass validated data through value check to set out of range values to NaN
ehresp.income_185_pov = ehresp.income_185_pov.apply(value_check, args=(1,2,True))

In [154]:
# Pass validated data through value check to set out of range values to NaN: income_130_pov
ehresp.income_130_pov = ehresp.income_130_pov.apply(value_check, args=(1,2,True))

# Export cleaned, engineered, set

In [155]:
# Combine all data
df_join_2 = ehresp.join(df_join_1)
display(df_join_2.shape, df_join_2.head(3))

(12708, 20)

,snap_benefits,general_health,height_in,income_130_pov,statistical_weight,num_children,income_185_pov,meal_preparer,grocery_shopper,secondary_eating_duration,secondary_drinking_duration,primary_eating_duration,secondary_drinking_day_prior,weight_lb,secondary_eating_day_prior,bmi,secondary_eating_all_day,secondary_drinking_all_day,activity_list,assisted_meals
case_id,,,,,,,,,,,,,,,,,,,,
20080101080013,0.0,3,67,NaN,1.739920e+07,0,1.0,1.0,1.0,0,0,20,0.0,180,0.0,28.2,0,0,NaN,NaN
20080101080015,0.0,2,68,NaN,5.435524e+06,0,NaN,3.0,3.0,0,0,5,0.0,200,0.0,30.4,0,0,NaN,NaN
20080101080088,0.0,2,67,2.0,2.925273e+06,0,2.0,1.0,1.0,0,0,90,0.0,100,0.0,15.7,0,0,NaN,NaN


In [156]:
# Append year
df_join_2['year'] = year

In [157]:
df_join_2.to_csv(path_or_buf=output_file_name)